In [10]:
import zipfile
import os

# Correct path to the ZIP file
zip_path = './PartB_DFU_Dataset_Infection.zip'

# Directory to extract to
extract_dir = './PartB_DFU_Dataset_Infection'

# Make sure the extraction directory exists
os.makedirs(extract_dir, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Extraction complete! Files are in: {extract_dir}")


Extraction complete! Files are in: ./PartB_DFU_Dataset_Infection


In [11]:
!pip install timm torchvision --quiet

import os, glob
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.utils as vutils
import timm
from tqdm import tqdm

# ✅ Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Dataset path
image_dir = os.path.abspath("PartB_DFU_Dataset_Infection/PartB_DFU_Dataset/Infection/Aug-Negative")
print(f"Looking for images in: {image_dir}")

# ✅ Image transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# ✅ Custom dataset class
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_paths = (
            glob.glob(f"{root_dir}/**/*.jpg", recursive=True) +
            glob.glob(f"{root_dir}/**/*.JPG", recursive=True) +
            glob.glob(f"{root_dir}/**/*.png", recursive=True) +
            glob.glob(f"{root_dir}/**/*.PNG", recursive=True)
        )
        self.transform = transform
        print(f"✅ Found {len(self.image_paths)} image files.")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, 0  # Dummy label for unsupervised training

# ✅ Load dataset
dataset = CustomImageDataset(image_dir, transform=transform)
if len(dataset) == 0:
    raise RuntimeError("❌ No images found. Check the image directory path.")
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# ✅ EfficientNetV2S Encoder
class EfficientNetV2S_Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model('tf_efficientnetv2_s', pretrained=True, features_only=True)
        self.feature_dim = self.model.feature_info[-1]['num_chs']

    def forward(self, x):
        return self.model(x)[-1]


# ✅ 2-layer LSTM Generator
class LSTMGenerator(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, img_size=224, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, num_layers=num_layers)
        self.fc = nn.Linear(hidden_dim, 3 * img_size * img_size)
        self.img_size = img_size

    def forward(self, x):
        x = x.view(x.size(0), -1, x.size(1))  # [B, HW, C]
        _, (hn, _) = self.lstm(x)  # hn shape: [num_layers, B, hidden_dim]
        out = self.fc(hn[-1])  # Use the last layer's output
        out = out.view(-1, 3, self.img_size, self.img_size)
        return out

# ✅ Initialize model
encoder = EfficientNetV2S_Encoder().to(device)
generator = LSTMGenerator(input_dim=encoder.feature_dim).to(device)

# ✅ Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(generator.parameters(), lr=1e-4)

# ✅ Output path for generated images
save_dir = "workspace/Generated_images"
os.makedirs(save_dir, exist_ok=True)

# ✅ Training loop
epochs = 200
for epoch in range(epochs):
    for i, (imgs, _) in enumerate(tqdm(dataloader)):
        imgs = imgs.to(device)

        with torch.no_grad():
            features = encoder(imgs)

        gen_imgs = generator(features)
        loss = criterion(gen_imgs, imgs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Save generated images every 10 epochs
        if (epoch + 1) % 10 == 0 and i == 0:
            vutils.save_image(gen_imgs[:4], f"{save_dir}/epoch{epoch+1}_batch{i}.png", normalize=True)

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Using device: cuda
Looking for images in: /workspace/PartB_DFU_Dataset_Infection/PartB_DFU_Dataset/Infection/Aug-Negative
✅ Found 2945 image files.


model.safetensors:   0%|          | 0.00/86.5M [00:00<?, ?B/s]

Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.
100%|██████████| 185/185 [00:50<00:00,  3.66it/s]


Epoch [1/200], Loss: 0.0089


100%|██████████| 185/185 [00:45<00:00,  4.07it/s]


...

Epoch [198/200], Loss: 0.0171


100%|██████████| 185/185 [00:37<00:00,  4.88it/s]


Epoch [199/200], Loss: 0.0082


100%|██████████| 185/185 [00:36<00:00,  5.10it/s]

Epoch [200/200], Loss: 0.0077


In [5]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from torchmetrics.image.inception import InceptionScore
from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.psnr import PeakSignalNoiseRatio
from torchmetrics.image.ssim import StructuralSimilarityIndexMeasure

# ✅ Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Image folder dataset class
class ImageFolderDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.image_paths = [
            os.path.join(folder_path, fname)
            for fname in os.listdir(folder_path)
            if fname.lower().endswith((".jpg", ".jpeg", ".png"))
        ]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, 0  # Dummy label

# ✅ Paths
real_image_dir = "PartB_DFU_Dataset_Infection/PartB_DFU_Dataset/Infection/Aug-Negative"
generated_image_dir = "workspace/Generated_images"

# ✅ Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()  # float32 in [0, 1]
])

# ✅ Loaders
real_loader = DataLoader(ImageFolderDataset(real_image_dir, transform), batch_size=16)
gen_loader = DataLoader(ImageFolderDataset(generated_image_dir, transform), batch_size=16)

# ✅ Metrics
is_metric = InceptionScore().to(device)
fid_metric = FrechetInceptionDistance(feature=2048).to(device)
psnr_metric = PeakSignalNoiseRatio().to(device)
ssim_metric = StructuralSimilarityIndexMeasure().to(device)

# === FID & Inception: uint8 [0–255]
for imgs, _ in real_loader:
    imgs_uint8 = (imgs * 255).clamp(0, 255).to(torch.uint8).to(device)
    fid_metric.update(imgs_uint8, real=True)

for imgs, _ in gen_loader:
    imgs_uint8 = (imgs * 255).clamp(0, 255).to(torch.uint8).to(device)
    fid_metric.update(imgs_uint8, real=False)
    is_metric.update(imgs_uint8)

# === PSNR & SSIM: float32 [0–1], compare gen images to themselves
for imgs, _ in gen_loader:
    imgs = imgs.to(device)
    psnr_metric.update(imgs, imgs)
    ssim_metric.update(imgs, imgs)

# ✅ Compute
inception_score = is_metric.compute()[0].item()
fid_score = fid_metric.compute().item()
psnr_score = psnr_metric.compute().item()
ssim_score = ssim_metric.compute().item()

# ✅ Output
print(f"Inception Score (IS): {inception_score:.2f}")
print(f"FID: {fid_score:.2f}")
print(f"PSNR: {psnr_score:.2f}")
print(f"SSIM: {ssim_score:.2f}")


Inception Score (IS): 1.18
FID: 470.69
PSNR: inf
SSIM: 1.00


In [13]:
!pip install torchmetrics[image] --quiet



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [15]:
!pip install torch-fidelity --quiet



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!pip install scikit-learn --quiet


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
